# Generate popularity table

In [1]:
import sys
import os
try:
    import warnings                                                   
    warnings.filterwarnings('ignore')     
    import pandas as pd       
    import numpy as np
    import sqlite3
except ImportError:
    print("Error: missing one of the libraries..")
    sys.exit()



print(os.getcwd())
print(sys.argv[0])
print(os.path.dirname(os.path.realpath('__file__')))

/Users/marcelino/dmarcelinobr/pollingpoint/dashboards/notebooks
/Users/marcelino/miniconda3/envs/pymc3/lib/python3.8/site-packages/ipykernel_launcher.py
/Users/marcelino/dmarcelinobr/pollingpoint/dashboards/notebooks


## Load and clean the data

In [33]:
con = sqlite3.connect('../../pollingpoint.db')

file_long = pd.read_sql(""" SELECT * FROM aprovacao;""", con=con) 

con.close()

In [34]:
# converte para datetime 
file_long["data_fim"] = pd.to_datetime(file_long["data_fim"])

# converte para proporção 
file_long[["positiva", "regular", "negativa"]] = file_long[["positiva", "regular", "negativa"]].copy()/100

# Subset for the current president
data = file_long.loc[file_long.data_fim >= pd.to_datetime("2019-01-01")]

# Filter for data to show
df = data[['data_fim', 'empresa', 'tipo', 'amostra', 'positiva', 'regular', 'negativa', 'modo', 'erro']].reset_index(drop=True)

,data_fim,empresa,tipo,amostra,positiva,regular,negativa,modo,erro
0,2019-01-10,IDEIA Big Data,Avaliação do governo federal,NaN,0.500,0.280,0.220,Online,3.0
1,2019-01-15,Ipespe,Avaliação do governo federal,1000.0,0.400,0.290,0.200,CATI,3.2
2,2019-01-21,IDEIA Big Data,Avaliação do governo federal,NaN,0.510,0.280,0.210,Online,3.0
3,2019-01-30,IDEIA Big Data,Avaliação do governo federal,2300.0,0.500,0.280,0.220,Online,2.0
4,2019-01-30,Ibope,Avaliação do governo federal,2000.0,0.490,0.260,0.110,Presencial,2.0
...,...,...,...,...,...,...,...,...,...
352,2021-07-15,IDEIA Big Data,Aprovacao do governo federal,1200.0,0.260,0.210,0.480,CATI,3.0
353,2021-07-07,Ranking Brasil,Avaliação do governo federal,3000.0,0.260,0.202,0.511,CATI,2.5
354,2021-07-07,Ranking Brasil,Aprovação do governo federal,3000.0,0.342,NaN,0.631,CATI,2.5
355,2021-07-21,PoderData,Avaliação do governo federal,2500.0,0.260,0.150,0.560,IVR,2.0


## Sanity checks

We need to check that we haven't introduced any typos in the pollsters or methods, and do a quick visual check on the most recent approval values.

In [22]:
pd.crosstab(df.empresa, df.modo, margins=True)

modo,CATI,IVR,Online,Presencial,All
empresa,,,,,
Atlas,0,0,24,0,24
Datafolha,7,0,0,6,13
IDEIA Big Data,19,0,63,0,82
INCT/IDDC,1,0,0,2,3
Ibope,0,0,0,21,21
Ibpad,7,0,0,0,7
Instituto FSB,8,0,0,0,8
Instituto Methodus,1,0,0,0,1
Ipec,0,0,0,4,4


In [19]:
# Salve database to chart 
df.to_csv('exports/polls_popularity.csv', sep=',', encoding='utf-8')